In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger
from models import LeNetRegressor
from keras import backend as K
K.set_image_data_format('channels_last')

/home/nicoli/github/alexnet/data_generator/batch_generator.py:42: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression.csv')


# In[3]:


cols = list(df.columns[1:])
in_format = list(df.columns)


In [3]:
train_dataset = BatchGenerator(box_output_format=cols)
validation_dataset = BatchGenerator(box_output_format=cols)

train_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop',
                        input_format=in_format)

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_val_split_47950-70-10-20.csv', 
                             images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop',
                             input_format=in_format)

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 100

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32

In [5]:

train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         flip=0.5,
                                         equalize=True,
                                         divide_by_stddev=255,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   flip=0.5,
                                                   equalize=True,
                                                   divide_by_stddev=255,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())


Number of images in the dataset: 33565
Number of images in the dataset: 4795


In [6]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [7]:
lenet = LeNetRegressor(1, img_width, img_height, img_depth, ativacao='lrelu')

/home/nicoli/github/alexnet/models.py:78: UserWarning: Considering a regression task with output function being 'relu'
  warnings.warn('Considering a regression task with output function being \'relu\'')


In [8]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [11]:
csv_logger = CSVLogger('callbacks/lenet/age/history-regression-fase2-2.csv', append=True, separator=',')

checkpoint = ModelCheckpoint(filepath='callbacks/lenet/age/class-weights-reg-fase2-2.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=1)
callbacks=[checkpoint, csv_logger]

In [12]:
lenet.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [ ]:
history = lenet.model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)

Epoch 1/100
525/524 [==============================] - 1639s 3s/step - loss: 209.5940 - mean_absolute_error: 11.3658 - val_loss: 194.9050 - val_mean_absolute_error: 10.7846

Epoch 00001: saving model to callbacks/lenet/age/class-weights-reg-fase2-2.01-194.91.hdf5
Epoch 2/100
417/524 [======================>.......] - ETA: 4:46 - loss: 189.3671 - mean_absolute_error: 10.8180